<a href="https://colab.research.google.com/github/Mikhaelcmc/Assistente-de-Voz-Inteligente-Whisper-ChatGPT-gTTS/blob/main/Assistente_de_voz.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import speech_recognition as sr
import whisper
from openai import OpenAI
from gtts import gTTS
import os

# --- CONFIGURAÇÕES ---
client = OpenAI(api_key="SUA_CHAVE_AQUI")
modelo_whisper = whisper.load_model("base") # 'base' é rápido para tempo real
historico = [{"role": "system", "content": "Você é um assistente de voz conciso."}]

def capturar_e_responder():
    reconhecedor = sr.Recognizer()

    with sr.Microphone() as source:
        print("\n--- Ajustando ruído ambiente... Aguarde um segundo.")
        reconhecedor.adjust_for_ambient_noise(source, duration=1)
        print("Pode falar! Estou ouvindo...")

        try:
            # Captura o áudio
            audio_data = reconhecedor.listen(source, timeout=5, phrase_time_limit=10)

            # Salva o áudio temporariamente
            with open("temp_audio.wav", "wb") as f:
                f.write(audio_data.get_wav_data())

            # 1. WHISPER (Voz -> Texto)
            transcricao = modelo_whisper.transcribe("temp_audio.wav")
            texto_user = transcricao["text"]
            print(f"Você disse: {texto_user}")

            # 2. CHATGPT (Processamento)
            historico.append({"role": "user", "content": texto_user})
            response = client.chat.completions.create(
                model="gpt-3.5-turbo",
                messages=historico
            )
            resposta_ia = response.choices[0].message.content
            print(f"IA: {resposta_ia}")
            historico.append({"role": "assistant", "content": resposta_ia})

            # 3. gTTS (Texto -> Voz)
            tts = gTTS(text=resposta_ia, lang='pt')
            tts.save("resposta.mp3")

            # Reproduzir (Windows usa 'start', Mac 'afplay', Linux 'mpg123')
            os.system("start resposta.mp3")

        except sr.WaitTimeoutError:
            print("Não ouvi nada...")
        except Exception as e:
            print(f"Erro: {e}")

# Executar
if __name__ == "__main__":
    while True:
        capturar_e_responder()
        input("\nPressione Enter para falar novamente ou Ctrl+C para sair...")

ModuleNotFoundError: No module named 'speech_recognition'